In [ ]:
pip install rasterio geopandas scikit-learn numpy


### Import Relevant Packages 

In [1]:
import rasterio
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import geopandas as gpd
from rasterio.mask import mask
from rasterio.features import geometry_mask

### Load Sentinel-2 image

In [ ]:
sentinel_image_path = "/home/eoafrica/Sentinel2_AWbasin/outputs_rgb/stacked_rgb.tif"
with rasterio.open(sentinel_image_path) as src:
    sentinel_image = src.read()

###  Load ground truth GCP shapefile data

In [5]:
gcp_shapefile_path 

gcp_data = gpd.read_file(gcp_shapefile_path)

### Extract the values of the image pixels at the locations of the GCPs

In [6]:
gcp_points = gcp_data.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
gcp_values = []
for point in gcp_points:
    row, col = src.index(point[0], point[1])
    gcp_values.append(sentinel_image[:, row, col])

gcp_values = np.array(gcp_values)

###  Extract corresponding class labels from the GCP data

In [8]:
class_labels = gcp_data["class_labe"].values.astype(int)

###  Split data into training and testing sets

In [9]:
X_train, X_test, y_train, y_test = train_test_split(gcp_values, class_labels, test_size=0.3, random_state=42)

### Train Random Forest classifier

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

### Predict on the test set

In [11]:
y_pred = clf.predict(X_test)

### Calculate accuracy

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

###  Optionally, classify the whole image

In [ ]:
predicted_image = np.zeros_like(sentinel_image[0])
for i in range(sentinel_image.shape[1]):
    for j in range(sentinel_image.shape[2]):
        pixel_values = sentinel_image[:, i, j].reshape(1, -1)
        predicted_class = clf.predict(pixel_values)
        predicted_image[i, j] = predicted_class

### Save the classified image

In [ ]:
# Save the classified image
classified_image_path = "/home/eoafrica/Sentinel2_AWbasin/RF_LULCresult/classified_image.tif"  # Path to save the classified image
with rasterio.open(classified_image_path, 'w', **profile) as dst:
    dst.write(classified_image.astype(rasterio.uint8))